In [1]:
import torch

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [2]:
seed = 0
# Set the seed for reproducibility
torch.manual_seed(seed)

In [3]:
import torch
import gpytorch
from botorch.models import SingleTaskGP
from gpytorch.likelihoods import FixedNoiseGaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import MaternKernel, ScaleKernel

# Example Usage for 1D
dim = 4

# Define your kernel
nu = 2.5
lengthscale = 0.1
outputscale = 1.0
base_kernel = MaternKernel(nu=nu).double()
base_kernel.lengthscale = torch.tensor([[lengthscale]])
scale_kernel = ScaleKernel(base_kernel).double()
scale_kernel.outputscale = torch.tensor([[outputscale]])

# Define Noise Level
noise_level = 1e-4

# Initialize Placeholder Data with Correct Dimensions
num_samples = 1  # Replace with actual number of samples
num_features = dim  # Replace with actual number of features
train_X = torch.empty(num_samples, num_features)  # Placeholder data
train_Y = torch.empty(num_samples, 1)             # Placeholder data
Yvar = torch.ones(num_samples) * noise_level

# Initialize Model
model = SingleTaskGP(train_X, train_Y, likelihood = FixedNoiseGaussianLikelihood(noise=Yvar), covar_module=scale_kernel)

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([1.6779e+243]), std = tensor([nan])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


In [4]:
from botorch.sampling.pathwise import draw_kernel_feature_paths
matern_sample = draw_kernel_feature_paths(model, sample_shape=torch.Size([1]))
def objective_function(x):
    return matern_sample(x).squeeze(0).detach()

In [5]:
from botorch.utils.sampling import optimize_posterior_samples
# Find the global optimum
maximize = True
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
global_optimum_point, global_optimum_value = optimize_posterior_samples(paths=matern_sample, bounds=bounds, raw_samples=1024*dim, num_restarts=20*dim, maximize=maximize)

In [6]:
from gpytorch.kernels import MaternKernel, ScaleKernel
# Set up the kernel
base_kernel = MaternKernel(nu=nu).double()
base_kernel.lengthscale = lengthscale
base_kernel.raw_lengthscale.requires_grad = False
scale_kernel = ScaleKernel(base_kernel).double()
scale_kernel.outputscale = torch.tensor([[outputscale]])
scale_kernel.raw_outputscale.requires_grad = False

In [7]:
maximize = True
output_standardize = False

In [8]:
from botorch.utils.sampling import draw_sobol_samples
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from botorch.acquisition import LogExpectedImprovement

In [9]:
from pandora_bayesopt.acquisition.log_gittins import LogGittinsIndex
budget = 10
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Create an instance of BayesianOptimizer
logPBGI_optimizer = BayesianOptimizer( 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x,
        objective=objective_function, 
        output_standardize=output_standardize
    )
# Run the optimization
logPBGI_optimizer.run_until_budget(
            budget = budget, 
            acquisition_function_class = LogGittinsIndex,
            lmbda = 0.0001
        )
logPBGI_best_history = logPBGI_optimizer.get_best_history()

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.2982]), std = tensor([0.9299])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


mean: tensor([-0.3330,  0.1799, -0.4689, -0.0737, -0.2395],
       grad_fn=<SqueezeBackward1>)
u: tensor([-3.3619, -3.2872, -3.3638, -3.3397, -3.2996], requires_grad=True)
log_h_u: tensor([-9.2059, -8.9208, -9.2134, -9.1208, -8.9677])
log_h_u.grad_fn: None


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
from botorch.acquisition.analytic import _log_ei_helper
import torch
from torch.autograd import grad

# Define the input tensor
u_test = torch.tensor([-3.3619, -3.2872, -3.3638, -3.3397, -3.2996], requires_grad=True)

# Compute log_h_u
log_h_u_test = _log_ei_helper(u_test)

print("u_test.requires_grad:", u_test.requires_grad)
print("u_test.grad_fn:", u_test.grad_fn)
print("log_h_u_test.requires_grad:", log_h_u_test.requires_grad)
print("log_h_u_test.grad_fn:", log_h_u_test.grad_fn)

# Compute gradients for the vector
dlogh_du_test = grad(
    outputs=log_h_u_test,  # Pass the vector directly
    inputs=u_test,
    grad_outputs=torch.ones_like(log_h_u_test),  # Match batch size
    retain_graph=True,  # Retain graph for debugging if needed
    allow_unused=True,
)[0]

print("log_h_u_test:", log_h_u_test)
print("dlogh_du_test:", dlogh_du_test)  # Should not raise RuntimeError

u_test.requires_grad: True
u_test.grad_fn: None
log_h_u_test.requires_grad: True
log_h_u_test.grad_fn: <WhereBackward0 object at 0x2899ba400>
log_h_u_test: tensor([-9.2060, -8.9207, -9.2134, -9.1207, -8.9677], grad_fn=<WhereBackward0>)
dlogh_du_test: tensor([3.8538, 3.7870, 3.8555, 3.8339, 3.7981])
